# 프롬프트 엔지니어링을 위한 모범 사례

출처: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api


In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

# 1. 최신 모델 사용

최고의 결과를 위해 최신 모델을 사용하세요.

# 2. 프롬프트의 시작 부분에 지침을 배치하고 ### 또는 """로 지침과 컨텍스트를 구분하세요

In [ ]:
response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"당신은 도움이 되는 어시스턴트입니다."},
                                {"role":"user","content": '아래 텍스트를 가장 중요한 요점의 글머리표 목록으로 한글로 요약해 주세요. \n\n \
                ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
                capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
                raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
                broadly sharing benefits and the need to prioritize safety. \
                Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###',}],
                max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI와 Microsoft가 파트너십을 확장하기로 발표했습니다.
- Microsoft는 2019년과 2021년에 이어 다년간의 수십억 달러 규모의 투자를 합니다.
- 이러한 투자는 OpenAI가 독립적인 연구를 계속하고 점점 더 안전하고 유용하며 강력한 AI를 개발할 수 있도록 지원합니다.
- OpenAI는 모든 인류에게 혜택을 주는 고급 AI를 보장하겠다는 사명을 가지고 있습니다.
- OpenAI는 이익을 제한하는 구조를 가진 회사로, 비영리 단체에 의해 운영됩니다.
- 이러한 구조는 중요한 신념을 포기하지 않고도 사명을 수행하기 위한 자본을 조달할 수 있게 합니다.
- Microsoft는 이러한 비전과 가치를 공유하며, 이 파트너십은 발전에 있어 중요한 역할을 합니다.


# 3. 원하는 컨텍스트, 결과, 길이, 형식, 스타일 등에 대해 구체적이고 상세하게 작성하세요

In [6]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '경봉궁을 위한 시 한편 써줘',}],
        max_tokens=400,)

print(response.choices[0].message.content)


경복궁을 위한 시

고요한 새벽의 안개 속에서  
경복궁의 돌담은 이야기를 품고  
한 세기의 시간을 아우른다

웅장한 광화문을 지나  
푸른 기와 지붕 아래  
왕들의 발자취가 남아있네

자연과 어우러진 창덕궁 정원처럼  
경복궁의 마당엔 바람이 춤추고  
역사의 숨결을 담은 향기가 맴돈다

햇살에 빛나는 경회루의 물결이여  
그 속에 비친 우리의 지난날  
다시금 되돌아보게 하는 거울이 되어라

마주한 과거와 현재가  
여기서 조용히 손을 잡고  
미래를 향해 걸어가네

경복궁, 당신의 벽돌 하나하나가  
만년의 이야기를 타고  
오늘도 우리에게 속삭이니


In [7]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '경봉궁을 위한 시 한편 써줘 \
                특히 봄에서 여름으로 가는 게절에 아름다운 풍경을 충분히 묘사해줘',}],
        max_tokens=400,)

print(response.choices[0].message.content)



경복궁의 봄에서 여름으로 가는 길,

꽃잎은 나비처럼 춤을 추며 흩날리고,
연못의 잔잔한 물결이 빛을 받아 반짝인다.
분홍빛 살구나무가 은은한 향기를 풍기고,
왕벚나무 아래로 펼쳐진 길은 푸르게 물든다.

봄의 기운은 여름의 문턱에서 머물고,
푸른 하늘 아래로 왕조의 역사가 숨쉬고 있다.
긴 처마 아래로 비치는 햇살은 금빛을 띠고,
고요한 전각은 자연의 소리에 귀 기울인다.

연녹색 잎사귀가 바람에 속삭이며 흔들릴 때,
궁궐의 정원은 생명의 물결로 가득 찬다.
오랜 세월을 간직한 돌담은 새로이 빛을 받아
계절의 변화를 담담히 배웅한다.

찬란한 자연의 옷을 입은 경복궁은,
역사와 현대가 만나고 조화를 이루는 곳.
이곳에서의 봄은 마침내 여름으로 나아가며,
모든 순간이 아름다움으로 채워진다.


# 4. 원하는 출력 형식을 예제를 통해 명확히 표현하세요 (예제 1, 예제 2).

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '아래 텍스트에서 회사명 다음 연도를 추출하고 각 엔티티의 시작 인덱스와 끝 인덱스를 출력합니다.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34} \
                ###\
                OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 전하게 되어 기쁩니다. \
                Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년에 이루어진 이전 투자에 이은 것입니다. \
                2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 계속하고 더욱 안전하고 유용하며 강력한 \
                더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. \n\n \
                ###\
                ',}],
                
        max_tokens=400,)

print(response.choices[0].message.content)



To extract the company names and the years mentioned in the text along with their start and end indices, we can analyze the text as follows:

The text contains references to the following company names and years:
1. "OpenAI" is mentioned once.
2. "Microsoft" is also mentioned once.
3. The years mentioned are "2019" and "2021".

Here's the information with their start and end indices:

1. **OpenAI**
   - Text: "OpenAI"
   - Start Index: 0
   - End Index: 6

2. **Microsoft**
   - Text: "Microsoft"
   - Start Index: 9
   - End Index: 18

3. **2019**
   - Text: "2019"
   - Start Index: 72
   - End Index: 76

4. **2021**
   - Text: "2021"
   - Start Index: 81
   - End Index: 85

Therefore, the extracted entities with their indices are:

```json
[
    {"text": "OpenAI", "start": 0, "end": 6},
    {"text": "Microsoft", "start": 9, "end": 18},
    {"text": "2019", "start": 72, "end": 76},
    {"text": "2021", "start": 81, "end": 85}
]
```


In [14]:
prompt = """
                아래 텍스트에 언급된 엔티티를 추출합니다. 
                아래 텍스트에 언급된 중요한 엔티티를 추출합니다. 
                먼저 모든 회사 이름을 추출한 다음 모든 연도를 추출합니다, 
                그런 다음 콘텐츠에 맞는 특정 주제를 추출하고 마지막으로 일반적인 주요 주제를 추출합니다.\n\n 
                Desired format: 
                Company names: <comma_separated_list_of_company_names> 
                Years: -||- 
                Specific topics: -||- 
                General themes: -||- 

                ###\n\n
                OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 알려드리게 되어 기쁩니다.
                Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년 투자에 이어 
                2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
                더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. \n\n

        """

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)



Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: 파트너십 연장, 다년간 투자, 독립적인 연구, AI 개발 
General themes: 투자, 기술 협력, 안전하고 유용한 AI


# 5. 제로샷(zero-shot)으로 시작하고, 이후 몇 가지 예제를 제공하세요. 둘 다 효과가 없으면 미세 조정을 시도하세요 (업데이트 필요)

In [15]:
prompt = """
        OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 알려드리게 되어 기쁩니다.
        Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년 투자에 이어 
        2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
        더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다.

"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, 파트너십, 연장, 투자, 다년간, 수십억 달러, 독립 연구, AI 개발, 안전, 유용, 강력


In [17]:
system_prompt = """

        당신은 유용한 조수입니다. 아래 해당 텍스트에서 키워드를 추출하세요..
        텍스트: 스트라이프는 웹 개발자가 웹 사이트와 모바일 애플리케이션에 결제 처리를 통합하는 데 사용할 수 있는 API를 제공합니다. 
        키워드: 스트라이프, 결제 처리, API, 웹 개발자, 웹사이트, 모바일 애플리케이션 
        ###
        텍스트: OpenAI는 텍스트를 이해하고 텍스트를 생성하는 데 매우 능숙한 최첨단 언어 모델을 학습시켰습니다.
        이해하고 텍스트를 생성하는 데 매우 능숙한 최첨단 언어 모델을 학습시켰습니다. API를 통해 이러한 모델에 액세스할 수 있으며, 언어 처리와 관련된 거의 모든 작업을
        언어 처리와 관련된 모든 작업을 해결하는 데 사용할 수 있습니다.
        키워드: 언어 모델, 텍스트 처리, API.
        
"""

user_prompt = """
        텍스트: OpenAI와 Microsoft가 파트너십을 연장한다는 소식을 전하게 되어 기쁘게 생각합니다.
        Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년에 이루어진 이전 투자에 이은 것입니다.
        2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
        더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. 
        키워드:
"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content": system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, 파트너십, 투자, 독립적 연구, 안전한 AI, 강력한 AI.


# 6. 모호하고 부정확한 설명을 줄이세요

In [18]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '새 제품에 대한 설명을 작성하세요. 이 제품은 차세대 카시트입니다. 이 제품에 대한 설명은 몇 문장으로만 짧게 작성하고 너무 길지 않아야 합니다.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



안전과 편안함을 최우선으로 설계된 차세대 카시트입니다. 혁신적인 충격 흡수 기술과 조절 가능한 맞춤형 피팅 기능을 통해 아이에게 최적의 보호를 제공합니다. 또한, 쉬운 설치와 스타일리시한 디자인으로 어떤 차량에서도 완벽하게 어울립니다.


In [19]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '새 제품에 대한 설명을 작성하세요. 이 제품은 차세대 카시트입니다. 3~5문장으로 구성된 단락을 사용하여 이 제품을 설명하세요.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



이 차세대 카시트는 안전성과 편안함을 최우선으로 고려하여 설계되었습니다. 첨단 충격 흡수 기술을 적용하여 충돌 시 아이를 최적으로 보호하며, 조절 가능한 헤드레스트와 메모리 폼 쿠션으로 최적의 편안함을 제공합니다. 또한, 쉬운 설치와 탈착을 위한 스마트 잠금 시스템이 있어 부모님들에게 큰 편리함을 제공합니다. 이 카시트는 360도 회전 기능을 갖추어 아이를 차에 태우고 내릴 때의 불편함을 최소화했습니다. 마지막으로, 직관적인 디자인과 다양한 색상 옵션으로 부모와 아이 모두를 만족시키는 스타일리시한 외관을 자랑합니다.


# 7. 하지 말아야 할 것을 말하는 대신, 해야 할 것을 명확히 설명하세요

In [20]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. Customer: I can’t log in to my account.\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Let's try to identify the issue. Are you receiving any specific error messages when you attempt to log in?


In [21]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. \
                The agent will attempt to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear you're having trouble logging in to your account. Let's try to resolve this issue together. First, please ensure that you are entering the correct username and password. Check for any typing errors and ensure that your Caps Lock key is off, as passwords are case-sensitive.

If you're still unable to log in, I recommend visiting our help article at www.samplewebsite.com/help/faq for more detailed troubleshooting steps and guidance on resetting your password.

Additionally, if you're experiencing any specific error messages, please let me know, and I'll do my best to help you resolve them.


# 8. 코드 생성 관련 - 특정 패턴으로 모델을 유도하기 위해 "선도 단어"를 사용하세요

In [22]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# 다음과 같은 간단한 파이썬 함수를 작성합니다. \n\
                # 1. 마일 단위로 숫자를 요청하세요.\n\
                # 2. 마일을 킬로미터로 변환합니다.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


마일을 킬로미터로 변환하는 간단한 파이썬 함수를 작성할 수 있습니다. 일반적으로 1 마일은 약 1.60934 킬로미터에 해당합니다. 사용자의 입력을 받아서 이를 변환한 후 결과를 출력하도록 하겠습니다.

다음 코드를 살펴보세요:

```python
def convert_miles_to_kilometers():
    # 1. 사용자에게 마일 단위의 숫자를 입력받습니다.
    miles = float(input("마일을 입력하세요: "))
    
    # 2. 마일을 킬로미터로 변환합니다.
    kilometers = miles * 1.60934
    
    # 3. 결과를 출력합니다.
    print(f"{miles} 마일은 {kilometers:.2f} 킬로미터입니다.")

# 함수를 실행합니다.
convert_miles_to_kilometers()
```

위의 코드를 실행하면 사용자에게 마일 단위의 숫자를 입력하도록 요청하고, 입력된 값을 킬로미터로 변환하여 결과를 출력합니다. `:.2f` 형식을 사용하여 변환된 킬로미터 값을 소수점 둘째 자리까지 표시합니다.
